In [5]:
#!pip install transformers sentence-transformers faiss-cpu

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [10]:


# Simulando uma base de dados de documentos
documents = [
    "A energia renovável é obtida de recursos naturais que se regeneram naturalmente.",
    "Os principais tipos de energia renovável são solar, eólica, hidroelétrica e biomassa.",
    "A energia renovável ajuda a reduzir a emissão de gases de efeito estufa.",
    "Investir em energia renovável pode impulsionar a economia verde."
]

# Passo 1: Indexação dos documentos usando embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')
document_embeddings = embedder.encode(documents)

# Criação do índice FAISS
d = document_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(document_embeddings)

# Função para recuperar documentos relevantes
def retrieve_documents(query, k=2):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, k)    
    retrieved_docs = [documents[idx] for idx in indices[0]]
    return retrieved_docs

# Carregando o modelo gerador (usando T5 para resumo)
tokenizer = AutoTokenizer.from_pretrained('t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

# Função para gerar resposta
def generate_answer(question, retrieved_docs):
    context = " ".join(retrieved_docs)
    input_text = f"summarize: question: {question} context: {context}"    
    inputs = tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=150, num_beams=5, early_stopping=True)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Exemplo de uso
if __name__ == "__main__":
    question = "Pode falar sobre energia renovável, além de efeito estufa?"
    retrieved_docs = retrieve_documents(question)
    answer = generate_answer(question, retrieved_docs)
    print("Pergunta:", question)
    print("Resposta:", answer)

Pergunta: Pode falar sobre energia renovável, além de efeito estufa?
Resposta: em energia renovável pode impulsionar a economia verde


Esta arquitetura combina recuperação de informações com geração de texto para produzir respostas mais informadas e contextualizadas.

1. Base de Dados:
   - Simulada por uma lista de strings chamada `documents`.
   - Cada string representa um documento com informações sobre energia renovável.

2. Indexação e Recuperação:
   - Utiliza o modelo `SentenceTransformer('all-MiniLM-L6-v2')` para criar embeddings dos documentos.
   - Os embeddings são armazenados em um índice FAISS (Facebook AI Similarity Search) para busca eficiente.
   - A função `retrieve_documents()` realiza a recuperação:
     - Converte a query em um embedding.
     - Usa o índice FAISS para encontrar os k documentos mais similares.

3. Geração de Resposta:
   - Utiliza o modelo T5 ('t5-small') para gerar respostas.
   - A função `generate_answer()`:
     - Combina a pergunta e os documentos recuperados em um único texto de entrada.
     - Usa o tokenizer T5 para preparar os dados para o modelo.
     - Gera uma resposta usando o modelo T5.

4. Fluxo de Execução:
   a. O usuário faz uma pergunta.
   b. A pergunta é passada para `retrieve_documents()` para encontrar documentos relevantes.
   c. Os documentos recuperados e a pergunta são passados para `generate_answer()`.
   d. O modelo T5 gera uma resposta baseada na pergunta e nos documentos recuperados.

5. Componentes Principais:
   - Embedder: SentenceTransformer para criar representações vetoriais dos textos.
   - Índice: FAISS para busca eficiente de similaridade.
   - Gerador: Modelo T5 para geração de texto.
   - Tokenizer: Prepara os dados para o modelo T5.

6. Particularidades:
   - Usa "summarize:" como prefixo na entrada do T5, orientando-o a gerar um resumo.
   - Limita o tamanho máximo da entrada e da saída para evitar problemas com sequências muito longas.
   - Utiliza beam search (num_beams=5) para melhorar a qualidade da geração.

Esta arquitetura permite que o sistema combine informações de múltiplos documentos relevantes para gerar uma resposta coerente e informativa, superando as limitações de modelos de linguagem treinados apenas com conhecimento estático.